In [ ]:
!pip install -U "ray[air]"

# Training

In [ ]:
import boto3
import pickle
import ray
from ray.air.config import RunConfig, ScalingConfig
from ray.train.xgboost import XGBoostTrainer

In [ ]:
ray.shutdown()
ray.init(address="ray://ray-cluster-kuberay-head-svc.ray-cluster.svc.cluster.local:10001")

In [ ]:
ray.cluster_resources()

In [ ]:
bucket = "fm-ops-datasets"
prefix = "model"

def prepare_dataset():
    dataset = ray.data.read_csv(f"s3://{bucket}/training/demo.csv")

    train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)
    test_dataset = valid_dataset.drop_columns(cols=["Target"])

    return train_dataset, valid_dataset, test_dataset

In [ ]:
train_dataset, valid_dataset, test_dataset = prepare_dataset()

In [ ]:
trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(
        num_workers=2,
        use_gpu=False,
        _max_cpu_fraction_per_node=0.9,
    ),
    run_config=RunConfig(
        name="training_demo", storage_path=f"s3://{bucket}/{prefix}"
    ),
    label_column="Target",
    num_boost_round=20,
    params={
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    datasets={"train": train_dataset, "valid": valid_dataset},
)

In [ ]:
model = trainer.fit()

In [ ]:
model.metrics

In [ ]:
checkpoint = model.checkpoint

In [ ]:
pickle_obj = pickle.dumps(checkpoint)
s3_resource = boto3.resource("s3")
s3_resource.Object(bucket, f"{prefix}/model.pkl").put(Body=pickle_obj)

# Serving

In [ ]:
import boto3
import pickle
from ray.train.xgboost import XGBoostPredictor
from ray import serve
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import pandas_read_json

In [ ]:
s3 = boto3.client("s3", region_name="us-east-1")
bucket = "fm-ops-datasets"
model_prefix = "model"

In [ ]:
response = s3.get_object(Bucket=bucket, Key=f"{model_prefix}/model.pkl")
pickle_data = response["Body"].read()
model_trainer = pickle.loads(pickle_data)

In [ ]:
model_trainer

In [ ]:
serve.run(
    PredictorDeployment.options(name="XGBoostService").bind(
        XGBoostPredictor, model_trainer, http_adapter=pandas_read_json
    )
)

# Predict

In [ ]:
import requests
import json

sample_request_input = {
    "Pregnancies": 6,
    "Glucose": 148,
    "BloodPressure": 72,
    "SkinThickness": 35,
    "Insulin": 0,
    "BMI": 33.6,
    "DiabetesPedigree": 0.625,
    "Age": 50,
}

response = requests.post(
    "http://localhost:8000/", json=[sample_request_input]
).json()

print(response)

# Cleanup

In [ ]:
# serve.delete("default")

In [ ]:
# serve.shutdown()

In [ ]:
# ray.shutdown()